## 完整 py 檔見 `flask_api/app.py`

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import re
from pymongo import MongoClient
from flask import Flask, request, jsonify
from flask_cors import CORS

# self-definition package.module
# del myfun

In [12]:
path_root = os.path.join(os.getcwd(), "../")
path_key = os.path.join(path_root, './key/key.txt')
path_result = os.path.join(path_root, "./result")
json_contact = os.path.join(path_result, "./contact_all.json")

# database credential
conn_string = open(path_key, 'r').read()

In [3]:
search = {
    "shape": 1,
    "kind": 2,
    "renter": {
        "renter_role": "屋主",
        "renter_fname": "廖" ,
        "renter_title": "先生"
    },
    "sex": 1, 
    "contact": "0906378828",
    "regionid": 1
}
# (1,2,3) = (限男生,限女生,男女皆可(不限))
# python: None, json: null

In [4]:
search = {
    "shape": 1,
    "kind": 4,
    "renter": {
        "renter_role": "屋主",
        "renter_fname": "吳" ,
        "renter_title": "小姐"
    },
    "sex": 2, 
    "contact": "0906378828",
    "regionid": None
}

In [20]:
search = {
    "renter": {
        "renter_role": "屋主",
        "renter_fname": "吳" ,
        "renter_title": "小姐"
    },
}

In [21]:
def post_json(raw_input):
    raw_input['renter.renter_role'] = raw_input["renter"]["renter_role"]
    raw_input['renter.renter_fname'] = raw_input["renter"]["renter_fname"]
    raw_input['renter.renter_title'] = raw_input["renter"]["renter_title"]
    raw_input.pop("renter")
    raw_dict = {k:v for k,v in raw_input.items() if v is not None}
    dictlist = []
    for key, value in raw_dict.items():
        temp = {key: value}
        dictlist.append(temp)
    return dictlist

In [22]:
a = search.copy()
b = post_json(a)
print(b)

[{'renter.renter_role': '屋主'}, {'renter.renter_fname': '吳'}, {'renter.renter_title': '小姐'}]


In [13]:
cluster = MongoClient(conn_string)
db = cluster['db_cathay']
collection = db['cathay_search']

In [47]:
# query
results = collection.find({"$and": b}, {"_id":0})
result = []
for x in results:
    result.append(x)

In [40]:
re_post_id = []
for x in result:
    re_post_id = re_post_id + [{'post_id':x['post_id']}]

In [51]:
collection = db['meta_data']
results = collection.find({"$or": re_post_id}, {"_id":0})

In [52]:
result = []
for x in results:
    result.append(x)

In [53]:
result

[{'street_name': '研究院路一段',
  'photoNum': '8',
  'shape': '公寓',
  'condition': ['電視',
   '冰箱',
   '冷氣',
   '洗衣機',
   '熱水器',
   '第四台',
   '網路',
   'landpost',
   '可開火',
   '有公車',
   '沒有陽台',
   '可短期租賃',
   '床',
   '衣櫃',
   '桌子',
   '椅子'],
  'refreshtime': '2021-06-09 19:40:02',
  'cover': 'https://hp2.591.com.tw/house/active/2020/05/13/158933976074635900_800x600.water3.jpg',
  'living': ['百貨公司', '進便利商店', '傳統市場', '夜市', '公園綠地', '學校', '醫療機構'],
  'region_name': '台北市',
  'updatetime': '2021-05-31 10:51:47',
  'area': 7,
  'hasimg': 1,
  'houseage': 0,
  'layout': '',
  'floorInfo': '樓層：3/3',
  'alley_name': '',
  'post_id': 10793892,
  'section_name': '南港區',
  'address_img': '捷運站月租雅房/含傢俱水電管理費免押金',
  'price': '11500 元/月',
  'room': 0,
  'kind': '雅房',
  'browsenum_all': 2934,
  'renter': {'renter_role': '屋主', 'renter_fname': '吳', 'renter_title': '小姐'},
  'sex': '男女皆可(不限)',
  'contact': '0906378828'},
 {'street_name': '羅斯福路三段',
  'photoNum': '7',
  'shape': '公寓',
  'condition': ['冰箱',
   '冷氣',
  

In [ ]:
cluster.close()